In [3]:
# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import pickle

# Load the dataset
data = pd.read_csv('combined_data.csv')

# Convert categorical data to numerical data
le = LabelEncoder()
data['bike_station_name'] = le.fit_transform(data['name'])
data['weather_main'] = le.fit_transform(data['weather_main'])

# Split the data into features and labels
X = data.drop(['available_bikes'], axis=1)
y = data['available_bikes']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create and train the model
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
score = model.score(X_test, y_test)
print(f'Test score: {score}')

# Save the trained model object as a pickle file
with open('model.pkl', 'wb') as handle:
    pickle.dump(model, handle)

ValueError: could not convert string to float: 'ROYAL HOSPITAL'

In [4]:
# Import necessary libraries
from flask import Flask, jsonify, request
import pickle
import pandas as pd

# Load the trained model object
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Create Flask app
app = Flask(__name__)

# Create API endpoint


@app.route('/predict', methods=['GET'])
def predict():
    # Get input parameters
    bike_station_name = request.args.get('bike_station_name')
    bike_station_number = request.args.get('bike_station_number')
    datequery = request.args.get('datequery')
    dayquery = request.args.get('dayquery')
    hourquery = request.args.get('hourquery')
    temperature = request.args.get('temperature')
    wind_speed = request.args.get('wind_speed')
    pressure = request.args.get('pressure')
    humidity = request.args.get('humidity')
    weather_main = request.args.get('weather_main')

    # Create input dataframe
    input_data = pd.DataFrame({
        'bike_station_name': [bike_station_name],
        'bike_station_number': [bike_station_number],
        'datequery': [datequery],
        'dayquery': [dayquery],
        'hourquery': [hourquery],
        'temperature': [temperature],
        'wind_speed': [wind_speed],
        'pressure': [pressure],
        'humidity': [humidity],
        'weather_main': [weather_main]
    })

    # Convert categorical data to numerical data
    input_data['bike_station_name'] = le.transform(input_data['bike_station_name'])
    input_data['weather_main'] = le.transform(input_data['weather_main'])

    # Scale the input data
    input_data = scaler.transform(input_data)

    # Make prediction using the trained model
    prediction = model.predict(input_data)

    # Return the prediction as a JSON response
    return jsonify({'prediction': prediction.tolist()})